In [1]:
%%capture
!pip install folium matplotlib mapclassify

import pandas as pd
import geopandas as gpd
import os

FOLDER_PATH = '/content/drive/MyDrive/DSL SMART CITIES'

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Import NILs

In [3]:
nil = gpd.read_file(os.path.join(FOLDER_PATH, 'Processed','nil.geojson'))
nil = nil.to_crs(epsg=4326)
nil.explore()

#Import Stallo Bikeme


In [4]:
cs = gpd.read_file(os.path.join(FOLDER_PATH,'STALLO_BIKEME.txt'))
#cs['numero_col'] = cs['numero_col'].astype(int)
cs = cs.drop_duplicates()
cs.columns

Index(['id_amat', 'tipo', 'nome', 'gruppo_id', 'ordinanza', 'deroghe',
       'val_inizio', 'val_fine', 'delibera', 'tratto', 'note', 'cartello',
       'dissuasori', 'geometry'],
      dtype='object')

In [5]:
# Convert Timestamp columns to strings for visualization
for col in cs.columns:
    if pd.api.types.is_datetime64_any_dtype(cs[col]):
        cs[col] = cs[col].astype(str)

cs.explore(fill='val_inizio')

In [6]:
cs

,id_amat,tipo,nome,gruppo_id,ordinanza,deroghe,val_inizio,val_fine,delibera,tratto,note,cartello,dissuasori,geometry
0,1,AP,Corso Concordia,0,54785/2004 (Istituzione area pedonale urbana),"velocipedi, veicoli diretti alle proprietà pri...",NaT,2049-12-31 23:00:00+00:00,425/2004,lato civici dispari da piazza Tricolore a via ...,None,AP,None,"MULTIPOLYGON (((9.20715 45.46813, 9.20714 45.4..."
1,7,AP,Erculea - Rugabella,0,47205/1999 (Istituzione area pedonale urbana),veicoli diretti all'interno delle proprietà pr...,NaT,2049-12-31 23:00:00+00:00,None,None,None,AP,None,"MULTIPOLYGON (((9.19016 45.45945, 9.19027 45.4..."
2,9,AP,Bergamini,0,38491/1993 (area pedonale),"velocipedi, residenti",NaT,2049-12-31 23:00:00+00:00,None,tutta,accesso da via Larga,AP,None,"MULTIPOLYGON (((9.19439 45.46118, 9.19434 45.4..."
3,11,DT,Vicolo Laghetto,0,44088/1997,veicoli diretti all'interno delle prorietà pri...,NaT,2049-12-31 23:00:00+00:00,None,None,None,DT,None,"MULTIPOLYGON (((9.19498 45.46143, 9.19498 45.4..."
4,12,AP,San Calimero,0,44890/1998 (Istituzione area pedonale urbana),"velocipedi,veicoli diretti alle proprietà priv...",1998-03-08 23:00:00+00:00,2049-12-31 23:00:00+00:00,None,None,None,AP,None,"MULTIPOLYGON (((9.19254 45.45630, 9.19258 45.4..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
212,1109,AP,VIA EMILIO GOLA,0,29/06/2022 (delimitazione area pedonale),carico e scarico merci (feriali dalle 5:00 all...,2022-06-22 22:00:00+00:00,2049-12-31 23:00:00+00:00,1152/2012,via Gola tra Alzaia Naviglio Pavese e via Pichi,None,AP,Pilomat,"MULTIPOLYGON (((9.17710 45.44898, 9.17707 45.4..."
213,1111,AP,Ripa di Porta Ticinese tra via Paoli e via Val...,0,791/2022,"residenti e domiciliati, possessori di box o p...",2022-06-29 22:00:00+00:00,NaT,None,Ripa di Porta Ticinese tra via Paoli e via Val...,None,AP,None,"MULTIPOLYGON (((9.17143 45.45100, 9.17151 45.4..."
214,1113,ZTL,"SAN LUIGI, SCRIVIA, DON BOSCO",0,697/2021 (DISCIPLINA VIABILISTICA DELLA ZONA A...,"Transito e sosta : residenti e domiciliati, ca...",2021-05-03 22:00:00+00:00,NaT,None,None,None,ZTL,None,"MULTIPOLYGON (((9.21464 45.44442, 9.21459 45.4..."
215,1114,DT,piazza Angilberto II,0,1160/2022 (divieto di transito),veicoli diretti all’interno delle proprietà pr...,2022-10-27 22:00:00+00:00,NaT,None,None,None,None,pilomat,"MULTIPOLYGON (((9.21649 45.43716, 9.21631 45.4..."


# Space Join

In [7]:
bikeme = nil.sjoin(cs, how='left')
bikeme.head()

,ID_NIL,NIL,area_sqkm,Residenti,geometry,index_right,id_amat,tipo,nome,gruppo_id,ordinanza,deroghe,val_inizio,val_fine,delibera,tratto,note,cartello,dissuasori
0,1,duomo,2.341616,16608,"POLYGON ((9.19482 45.47201, 9.19473 45.47192, ...",26.0,37.0,AP,Pioppette,0,51891/2002 (Istituzione area pedonale urbana),carico e scarico merci (lun-sab dalle 8:00 all...,2002-08-06 22:00:00+00:00,2049-12-31 23:00:00+00:00,None,tutta,None,AP,None
0,1,duomo,2.341616,16608,"POLYGON ((9.19482 45.47201, 9.19473 45.47192, ...",34.0,90.0,AP,Corso di Porta Ticinese - Colonne di San Lorenzo,0,48169/2000 (Istituzione area pedonale urbana),"veicoli diretti alle proprieta' private,veicol...",NaT,2049-12-31 23:00:00+00:00,None,None,None,AP,None
0,1,duomo,2.341616,16608,"POLYGON ((9.19482 45.47201, 9.19473 45.47192, ...",168.0,1056.0,DT,VIA DEI FABBRI,0,62325/2009 (divieto di transito),veicoli diretti all'interno delle proprietà pr...,NaT,2049-12-31 23:00:00+00:00,None,None,None,DT,None
0,1,duomo,2.341616,16608,"POLYGON ((9.19482 45.47201, 9.19473 45.47192, ...",74.0,157.0,AP,Pio IV,0,52037/2002 (Istituzione area pedonale urbana),carico e scarico merci (lun-sab dalle 8:00 all...,2002-06-13 22:00:00+00:00,2049-12-31 23:00:00+00:00,None,None,None,AP,None
0,1,duomo,2.341616,16608,"POLYGON ((9.19482 45.47201, 9.19473 45.47192, ...",24.0,34.0,DT,Sant'Ambrogio - Lanzone,0,50724/2001,veicoli connnessi alle funzioni religiose,NaT,2049-12-31 23:00:00+00:00,None,intersezione Lanzone - Sant'Ambrogio,None,DT,None


In [8]:
bikeme = bikeme.groupby(['ID_NIL','NIL','area_sqkm'])['id_amat'].size().reset_index()

In [9]:
bikeme.head()

,ID_NIL,NIL,area_sqkm,id_amat
0,1,duomo,2.341616,79
1,10,stazione centrale - ponte seveso,1.556019,10
2,11,isola,1.322887,3
3,12,maciachini - maggiolina,1.674918,1
4,13,greco - segnano,1.768603,2


In [10]:
bikeme = bikeme.rename(columns={'id_amat':'bike_slots'})
bikeme.head()

,ID_NIL,NIL,area_sqkm,bike_slots
0,1,duomo,2.341616,79
1,10,stazione centrale - ponte seveso,1.556019,10
2,11,isola,1.322887,3
3,12,maciachini - maggiolina,1.674918,1
4,13,greco - segnano,1.768603,2


In [11]:
bikeme['bike_slots_per_sqkm'] = bikeme['bike_slots']/bikeme['area_sqkm']
bikeme.head()

,ID_NIL,NIL,area_sqkm,bike_slots,bike_slots_per_sqkm
0,1,duomo,2.341616,79,33.737391
1,10,stazione centrale - ponte seveso,1.556019,10,6.426658
2,11,isola,1.322887,3,2.267767
3,12,maciachini - maggiolina,1.674918,1,0.597044
4,13,greco - segnano,1.768603,2,1.130836


# OUTPUT

In [13]:
#bikeme.to_csv(os.path.join(FOLDER_PATH,'Processed', 'bikeme_slots.csv'))